Comparing team attributes, not teams

## Usable Code

In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd

import pickle

The following cell gathers URL ends that will be inputted into the subsequent cell in order to gather relevant data for EACH game that will be inputted into a Pandas DF object.

In [2]:
years = range(2011, 2020)
weeks = range(1,22)
#years = [2020]
#weeks = [21]
game_link_ends = []
for year in years:
    for week in weeks:
        url0 = 'https://www.pro-football-reference.com/years/{}/week_{}.htm'.format(year, week)
        response0 = requests.get(url0)
        page0 = response0.text
        soup0 = BeautifulSoup(page0)
        for link in soup0.find_all(class_="right gamelink"):
            url1 = link.findNext()
            game_link_ends0 = url1.get('href')  #.get() method and 
                                                #calling index by attr (url['href']) act the same!
            game_link_ends.append(game_link_ends0.strip())
#2020 season done separately since season incomplete
weeks = range(1,20)
for week in weeks:
        url0 = 'https://www.pro-football-reference.com/years/2020/week_{}.htm'.format(week)
        response0 = requests.get(url0)
        page0 = response0.text
        soup0 = BeautifulSoup(page0)
        for link in soup0.find_all(class_="right gamelink"):
            url1 = link.findNext()
            game_link_ends0 = url1.get('href')  #.get() method and 
                                                #calling index by attr (url['href']) act the same!
            game_link_ends.append(game_link_ends0.strip())

Get to the actual game pages where we can extract data we want.

In [3]:
url2 = 'https://www.pro-football-reference.com'
url_list = []
for game_link in game_link_ends:
    url = ''.join([url2, game_link])
    url_list.append(url)

In [4]:
#Sanity checks to make sure we get all the boxscore links & subsequent pages (267 games/year)
#print(*url_list, sep='\n')
print(len(url_list), len(game_link_ends))

2669 2669


Now that we are actually on the pages where we can grab info for a model, let's grab it!

In [6]:
def game_stat_dict(url):
    '''
    From Pro-Football-Reference link stub, request gamepage, parse with BeautifulSoup, and
    collect 
        - game total (target) 
        - traditional stats (features)
        - efficiency stats (features)
    Return total & stats as a dictionary.
    '''
    response = requests.get(url)
    page = response.text
    page = page.replace("<!--","").replace("-->","")
    soup = BeautifulSoup(page, 'html.parser')

    headers = ['home','away','total','tot_1st',
               'tot_rush_att','tot_rush_yds','tot_rush_tds',
               'tot_comp','tot_att','tot_pass_yds','total_pass_tds','total_int',
               'tot_sacks','tot_sack_yds','tot_net_pass_yds','total_tot_yds',
               'tot_fum','tot_fum_l','total_to','tot_pen','tot_pen_yds',
               'tot_third_conv','tot_thirds','tot_third_per',
               'tot_fourth_conv','tot_fourths','tot_fourth_per',
               'margin','tot_off_epa','tot_pass_epa','tot_rush_epa','tot_to_epa','tot_spec_epa']

#scrape teams & totals from scorebox at top of page
    teams = []
    team_scores = []
    table = soup.find('table', class_="linescore nohover stats_table no_freeze")
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        #cols = [ele.text.strip() for ele in cols]
        team_scores.append([ele for ele in cols[-1] if ele]) # Get rid of empty values
        teams.append([ele.text for ele in cols[1] if ele])
    home = teams[1][0]
    away = teams[0][0]
    total = sum([int(scores) for teams in team_scores for scores in teams]) #flatten list to be able to add

#scrape team stats box for traditional metrics
    data1 = []
    table1 = soup.find('table', class_="add_controls stats_table")
    table_body1 = table1.find('tbody')

    rows1 = table_body1.find_all('tr')
    for row in rows1:
        cols = row.find_all('td')
        #cols = [ele.text.strip() for ele in cols]
        data1.append([ele.text for ele in cols if ele]) # Get rid of empty values
#process traditional stats table
#total first downs
    total_first_downs = sum(int(item) for item in data1[0])
#separate rush stats & make appropriate transformations
    rush_stats = [item.split('-') for item in data1[1]]
    total_rush_att = int(rush_stats[0][0]) + int(rush_stats[1][0])
    total_rush_yds = int(rush_stats[0][1]) + int(rush_stats[1][1])
    total_rush_tds = int(rush_stats[0][2]) + int(rush_stats[1][2])
#separate pass stats & make appropriate transformations
    pass_stats = [item.split('-') for item in data1[2]]
    total_comp = int(pass_stats[0][0]) + int(pass_stats[1][0])
    total_att = int(pass_stats[0][1]) + int(pass_stats[1][1])
    total_pass_yds = int(pass_stats[0][2]) + int(pass_stats[1][2])
    total_pass_tds = int(pass_stats[0][3]) + int(pass_stats[1][3])
    total_int = int(pass_stats[0][4]) + int(pass_stats[1][4])
#separate sack stats & make appropriate transformations
    sack_stats = [item.split('-') for item in data1[3]]
    total_sacks = int(sack_stats[0][0]) + int(sack_stats[1][0])
    total_sack_yds = int(sack_stats[0][1]) + int(sack_stats[1][1])
#net pass yards & total yards (stats don't require splits)
    total_net_pass_yds = int(data1[4][0]) + int(data1[4][1])
    total_tot_yds = int(data1[5][0]) + int(data1[5][1])
#separate fumbles & make appropriate transformations
    fum_stats = [item.split('-') for item in data1[6]]
    total_fum = int(fum_stats[0][0]) + int(fum_stats[1][0])
    total_fum_l = int(fum_stats[0][1]) + int(fum_stats[1][1])
#turnover stats (doesn't require split)
    total_to = int(data1[7][0]) + int(data1[7][1])
#separate penalty stats & make appropriate transformations
    pen_stats = [item.split('-') for item in data1[8]]
    total_pen = int(pen_stats[0][0]) + int(pen_stats[1][0])
    total_pen_yds = int(pen_stats[0][1]) + int(pen_stats[1][1])
#separate 3rd down stats & make appropriate transformations
    third_dn_stats = [item.split('-') for item in data1[9]]
    total_third_conv = int(third_dn_stats[0][0]) + int(third_dn_stats[1][0])
    total_thirds = int(third_dn_stats[0][1]) + int(third_dn_stats[1][1])
    total_third_per = round(100*total_third_conv/total_thirds,2)
#separate 4th down stats & make appropriate transformations
    fourth_dn_stats = [item.split('-') for item in data1[10]]
    total_fourth_conv = int(fourth_dn_stats[0][0]) + int(fourth_dn_stats[1][0])
    total_fourths = int(fourth_dn_stats[0][1]) + int(fourth_dn_stats[1][1])
    if total_fourths != 0:
        total_fourth_per = round(100*total_fourth_conv/total_fourths,2)
    else:
        total_fourth_per = None

#scrape team stats box for efficiency metrics
    data2 = []
    table2 = soup.find('table', id='expected_points')
    table_body2 = table2.find('tbody')

    rows2 = table_body2.find_all('tr')
    for row in rows2:
        cols = row.find_all('td')
        data2.append([ele.text for ele in cols if ele]) # Get rid of empty values
#grab relevant OFFESNIVE values & make appropriate transformations (defensive numbers are flipped sign)
    scoring_margin = int(abs(float(data2[0][0])))
    total_off_epa = round(float(data2[0][1]) + float(data2[1][1]),2)
    total_pass_epa = round(float(data2[0][2]) + float(data2[1][2]),2)
    total_rush_epa = round(float(data2[0][3]) + float(data2[1][3]),2)
    total_to_epa = round(float(data2[0][4]) + float(data2[1][4]),2)
    total_spec_epa = abs(float(data2[0][9]))

#make a dictionary of all the stats pulled from the gamepage
    game_dict = dict(zip(headers,[home,away,total,total_first_downs,
                                  total_rush_att,total_rush_yds,total_rush_tds,
                                  total_comp,total_att,total_pass_yds,total_pass_tds,total_int,
                                  total_sacks,total_sack_yds,total_net_pass_yds,total_tot_yds,
                                  total_fum,total_fum_l,total_to,total_pen,total_pen_yds,
                                  total_third_conv,total_thirds,total_third_per,
                                  total_fourth_conv,total_fourths,total_fourth_per,
                                  scoring_margin,total_off_epa,total_pass_epa,total_rush_epa,
                                  total_to_epa,total_spec_epa]))
    return game_dict

Make a list of the game stat dictionaries:

In [ ]:
game_list = []

for url in url_list:
    game_list.append(game_stat_dict(url))

Turn the list of dictionaries into a Pandas DataFrame object:

In [ ]:
game_df = pd.DataFrame(game_list) #convert list of dicts to DF
game_df.head() #sanity check that we got all games and data

In [ ]:
#sanity check that we got all the games and all the info we wanted
game_df.shape

Save pd object as a pickle file to load in our Regression notebook:

In [ ]:
with open('game_df.pickle', 'wb') as to_write:
    pickle.dump(game_df, to_write)

Other sanity checks:

In [ ]:
#sanity check that all games went through the function
print(len(game_list), len(url_list))

In [ ]:
#sanity check link
#url_list[31]

In [ ]:
#sanity check stats
#game_list[31]

In [ ]:
#sanity check on dictionary key-value pairs
headers = ['home','away','total','tot_1st',
           'tot_rush_att','tot_rush_yds','tot_rush_tds',
               'tot_comp','tot_att','tot_pass_yds','total_pass_tds','total_int',
               'tot_sacks','tot_sack_yds','tot_net_pass_yds','total_tot_yds',
               'tot_fum','tot_fum_l','total_to','tot_pen','tot_pen_yds',
               'tot_third_conv','tot_thirds','tot_third_per',
               'tot_fourth_conv','tot_fourths','tot_fourth_per',
               'margin','tot_off_epa','tot_pass_epa','tot_rush_epa','tot_to_epa','tot_spec_epa']
stats = [home, away,total,total_first_downs,
         total_rush_att,total_rush_yds,total_rush_tds,
         total_comp,total_att,total_pass_yds,total_pass_tds,total_int,
         total_sacks,total_sack_yds,total_net_pass_yds,total_tot_yds,
         total_fum,total_fum_l,total_to,total_pen,total_pen_yds,
         total_third_conv,total_thirds,total_third_per,
         total_fourth_conv,total_fourths,total_fourth_per,
         scoring_margin,total_off_epa,total_pass_epa,total_rush_epa,
         total_to_epa,total_spec_epa]

len(headers) - len(stats)

## Scratch Work Area

### Workable Page

In [ ]:
url_list[0]

In [ ]:
#start with first game to get a sense of basic structure; REMOVE COMMENTS
response = requests.get(url_list[0])
page = response.text
page = page.replace("<!--","").replace("-->","")
soup = BeautifulSoup(page, 'html.parser')

### Grab Target

In [ ]:
#scrape teams & totals from scorebox at top of page
teams = []
data = []
table = soup.find('table', class_="linescore nohover stats_table no_freeze")
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    #cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols[-1] if ele]) # Get rid of empty values
    teams.append([ele.text for ele in cols[1] if ele])
    
total = sum([int(item) for sublist in data for item in sublist])
total
away = teams[0][0]
home = teams[1][0]
print(home, away, total)

In [ ]:
data.append([ele.text for ele in cols[1] if ele])

### Grab Traditional Stats Table

In [ ]:
#scrape team stats box for traditional metrics
data1 = []
table1 = soup.find('table', class_="add_controls stats_table")
table_body1 = table1.find('tbody')

rows1 = table_body1.find_all('tr')
for row in rows1:
    cols = row.find_all('td')
    #cols = [ele.text.strip() for ele in cols]
    data1.append([ele.text for ele in cols if ele]) # Get rid of empty values

data1

### Process Traditional Stats Table

In [ ]:
#total first downs
total_first_downs = sum(int(item) for item in data1[0])
print(total_first_downs)

In [ ]:
#separate rush stats and make appropriate transformations
rush_stats = [item.split('-') for item in data1[1]]
total_rush_att = int(rush_stats[0][0]) + int(rush_stats[1][0])
print(total_rush_att)
total_rush_yds = int(rush_stats[0][1]) + int(rush_stats[1][1])
print(total_rush_yds)
total_rush_tds = int(rush_stats[0][2]) + int(rush_stats[1][2])
print(total_rush_tds)

In [ ]:
#separate pass stats and make appropriate transformations
pass_stats = [item.split('-') for item in data1[2]]
#print(pass_stats)
total_comp = int(pass_stats[0][0]) + int(pass_stats[1][0])
print(total_comp)
total_att = int(pass_stats[0][1]) + int(pass_stats[1][1])
print(total_att)
total_pass_yds = int(pass_stats[0][2]) + int(pass_stats[1][2])
print(total_pass_yds)
total_pass_tds = int(pass_stats[0][3]) + int(pass_stats[1][3])
print(total_pass_tds)
total_int = int(pass_stats[0][4]) + int(pass_stats[1][4])
print(total_int)

In [ ]:
#seperate sacks and make appropriate transformations
sack_stats = [item.split('-') for item in data1[3]]
#print(sack_stats)
total_sacks = int(sack_stats[0][0]) + int(sack_stats[1][0])
print(total_sacks)
total_sack_yds = int(sack_stats[0][1]) + int(sack_stats[1][1])
print(total_sack_yds)

In [ ]:
#net pass yards & total yards (stats that don't require splits)
total_net_pass_yds = int(data1[4][0]) + int(data1[4][1])
print(total_net_pass_yds)
total_tot_yds = int(data1[5][0]) + int(data1[5][1])
print(total_tot_yds)

In [ ]:
#seperate fumbles and make appropriate transformations
fum_stats = [item.split('-') for item in data1[6]]
print(fum_stats)
total_fum = int(fum_stats[0][0]) + int(fum_stats[1][0])
print(total_fum)
total_fum_l = int(fum_stats[0][1]) + int(fum_stats[1][1])
print(total_fum_l)

In [ ]:
#turnover stats (doesn't require split)
total_to = int(data1[7][0]) + int(data1[7][1])
print(total_to)

In [ ]:
#seperate penalities & make appropriate transformations
pen_stats = [item.split('-') for item in data1[8]]
print(pen_stats)
total_pen = int(pen_stats[0][0]) + int(pen_stats[1][0])
print(total_pen)
total_pen_yds = int(pen_stats[0][1]) + int(pen_stats[1][1])
print(total_pen_yds)

In [ ]:
#seperate 3rd downs & make appropriate transformations
third_dn_stats = [item.split('-') for item in data1[9]]
print(third_dn_stats)
total_third_conv = int(third_dn_stats[0][0]) + int(third_dn_stats[1][0])
print(total_third_conv)
total_thirds = int(third_dn_stats[0][1]) + int(third_dn_stats[1][1])
print(total_thirds)
total_third_per = round(100*total_third_conv/total_thirds,2)
print(total_third_per)

In [ ]:
#seperate 4th downs & make appropriate transformations
fourth_dn_stats = [item.split('-') for item in data1[10]]
print(fourth_dn_stats)
total_fourth_conv = int(fourth_dn_stats[0][0]) + int(fourth_dn_stats[1][0])
print(total_fourth_conv)
total_fourths = int(fourth_dn_stats[0][1]) + int(fourth_dn_stats[1][1])
print(total_fourths)
total_fourth_per = round(100*total_fourth_conv/total_fourths,2)
print(total_fourth_per)

In [ ]:
total_fourth_conv = 0
total_fourths = 0
if total_fourths != 0:
    total_fourth_per = round(100*total_fourth_conv/total_fourths,2)
else:
    total_fourth_per = None
print(total_fourth_per)

### Grab Efficiency Stats Table

In [ ]:
#scrape team stats box for efficiency metrics
data2 = []
table2 = soup.find('table', id='expected_points')
table_body2 = table2.find('tbody')

rows2 = table_body2.find_all('tr')
for row in rows2:
    cols = row.find_all('td')
    #cols = [ele.text.strip() for ele in cols]
    data2.append([ele.text for ele in cols if ele]) # Get rid of empty values

data2

### Process Efficiency Stats Table

In [ ]:
#grab relevant OFFESNIVE values & make appropriate transformations (defensive numbers are flipped sign)
scoring_margin = int(abs(float(data2[0][0])))
print(scoring_margin)
total_off_epa = round(float(data2[0][1]) + float(data2[1][1]),2)
print(total_off_epa)
total_pass_epa = round(float(data2[0][2]) + float(data2[1][2]),2)
print(total_pass_epa)
total_rush_epa = round(float(data2[0][3]) + float(data2[1][3]),2)
print(total_rush_epa)
total_to_epa = round(float(data2[0][4]) + float(data2[1][4]),2)
print(total_to_epa)
total_spec_epa = abs(float(data2[0][9]))
print(total_spec_epa)

## Scratch Work Graveyard

Cirlce back for:
- Play-by-play table to grab EPA's
- Another source for other advanced stats (CPOE)
- QB specific stats

In [ ]:
#Play-By-Play table, doesn't seem to pull in BS since it has weird "comment" section above it...
#will cirlce back if time permits
#REDEFINE DUMMY VARIABLES!!
data2 = []
table2 = soup.find('table', class_="sortable stats_table now_sortable sticky_table eq1 eq2 re2 le1")
table2
table_body = table.find('<tbody>')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data1.append([ele for ele in cols if ele]) # Get rid of empty values

In [ ]:
##Trying to grab totals from last row of Scoring Plays Table
#start with first game to get a sense of basic structure
#response = requests.get(url_list[0])
#page = response.text
#soup = BeautifulSoup(page, 'html.parser')
#Basic form to scrape any table
#data = []
#table = soup.find('table', attrs={'id':'scoring'})
#table_body = table.find('tbody')

#rows = table_body.find_all('tr')
#for row in rows:
 #   cols = row.find_all('td')
 #   cols = [ele.text.strip() for ele in cols]
 #   data.append([ele for ele in cols if ele]) # Get rid of empty values
#rows

In [ ]:
#Attempting to use Comment module from bs4 package (didn't work; claimed Comment wasn't defined??)
#to_remove = soup.find_all(text=Comment) 
#for element in to_remove: 
#    element.extract()

#for comments in soup.findAll(text=lambda text:isinstance(text, Comment)):
#    print(comments)
#    comments.extract()

#comments = soup.find_all(string=lambda text: isinstance(text, Comment))
#for c in comments:
#    print(c)
#    print("===========")
#    c.extract()
#soup